In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time

In [2]:
def get_data(soup, ember_id):
    container = soup.find_all("div", attrs={"id": ember_id})
    data = []
    if container is not None:
        for i in container[0].find_all('p') or container[0].find_all('ol'):
            data.append(i.text.strip("\n"))
    return ". ".join(data)

In [3]:
def parsetext(soup, numbers, dates, orgs, subjects, abstracts, events, lessons, recs, effs, relations, phases, missions, topics):
    dl = soup.find_all("dl", class_="dl-horizontal")
    
    if dl!=[]:
        cleaned_number_date_org = []
        for i in dl[0].find_all('dd'):
             cleaned_number_date_org.append(i.text)
        #finds report number (should be equal to n)
        number =  cleaned_number_date_org[0]
        numbers.append(number)
        #finds report date
        date =  cleaned_number_date_org[1]
        dates.append(date)
        #finds organization
        org =  cleaned_number_date_org[2]
        orgs.append(org)
    else: 
        return #lesson does not exist
    
    #finds subject
    container = soup.find_all("div", attrs= {'id':"ember509"})#, "class":"ember-view"})
    #print(container)
    subject = container[0].strong.text
    #print(subject)
    subjects.append(subject)
    
    #finds abstracts
    abstract = get_data(soup, "ember510"); abstracts.append(abstract)

    #finds events
    event = get_data(soup, "ember511"); events.append(event)
    
    #finds lessons
    lesson = get_data(soup, "ember512"); lessons.append(lesson)
    
    #finds recommendations
    rec = get_data(soup, "ember513"); recs.append(rec)
   
    #finds effectiveness of solutions
    #eff = get_data(soup, "ember514"); effs.append(eff)
    container = soup.find_all("div", attrs={"id": "ember514"})
    eff = ", ".join([container[0].text.replace("Evidence of Recurrence Control Effectiveness\n", "").strip("\n").strip("\n ")]); effs.append(eff)

    #finds program relation
    #relation = get_data(soup, "ember515"); relations.append(relation)
    container = soup.find_all("div", attrs={"id": "ember515"})
    relation = ", ".join([container[0].text.replace("Program Relation","").strip("\n").strip("\n ")]); relations.append(relation)
    
    #finds project/program phase
    container = soup.find_all("div", {'id':"ember516"})#, "class":"ember-view"})
    phase = ", ".join([container[0].text.replace("Program/Project Phase","").strip("\n").strip("\n ")]); phases.append(phase)
    
    em = soup.find('h3',text='Mission Directorate(s)').find_next_siblings()
    missions.append(", ".join([mission for mission in em[0].text.split("\n") if mission!=""]))

    em1 = soup.find('h3',text='Topic(s)').find_next_siblings()
    topics.append(", ".join([topic for topic in em1[0].text.split("\n") if topic!=""]))

In [4]:
numbers = []
dates = []
orgs = []
subjects = []
abstracts = []
events = []
lessons = []
recs = []
effs = []
relations = []
phases = []
missions = []
topics = []


In [5]:
test_lessons = [1,2,3,4,5,6,7,8,9,10,200,202,203,204,205,206,207,208,209,210,401,402,403,404,405,406,407,408,409,410, 1708,1709,1710,1401,1402,1403,1404,1405,1406]
full_lesson_numbers = pd.read_csv("LLIS_lesson_IDs.csv")
full_lessons = full_lesson_numbers['Lesson IDs'].tolist()
for n in test_lessons:
    url = "https://llis.nasa.gov/lesson/"+str(n)
    browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(1)
    respData = browser.page_source
    browser.quit()
    soup = BeautifulSoup(respData, 'html.parser')
    soup = soup.find_all("div", {"class":"ember-view"})
    soup = soup[0]
    parsetext(soup, numbers, dates, orgs, subjects, abstracts, events, lessons, recs, effs, relations, phases, missions, topics)


In [6]:
#data frame to store all the raw data
lessons_learned = pd.DataFrame({
    'Lesson ID': numbers,
    'Date Lesson Occurred': dates,
    'Submitting Organization': orgs,
    'Subject': subjects,
    'Abstract': abstracts,
    'Driving Event': events,
    'Lesson(s) Learned': lessons,
    'Recommendation(s)': recs,
    'Evidence': effs,
    'Program Relation': relations,
    'Program/project Phase': phases,
    'Mission Directorate(s)': missions,
    'Topics': topics 
}
)
display(lessons_learned)

,Lesson ID,Date Lesson Occurred,Submitting Organization,Subject,Abstract,Driving Event,Lesson(s) Learned,Recommendation(s),Evidence,Program Relation,Program/project Phase,Mission Directorate(s),Topics
0,1,1991-04-09,jsc,Implementation of Procedural Controls in Addit...,,During prelaunch check out of shuttle orbiter ...,Recurrence controls to prevent an original man...,Ensure that any process modifications are cons...,None,N/A,None,"Aeronautics Research, Science, Exploration S...",None
1,2,1989-10-17,jsc,Solder Defects Reduction by Training/certifica...,,"Following the shuttle mission STS-34, it was d...",Contamination of solder can result in connecti...,Training and certification of the solder techn...,None,N/A,None,"Aeronautics Research, Science, Exploration S...",None
2,3,1991-03-31,jsc,Hazard Potentials Associated With Maintenance ...,,While installing electrical cables into a cond...,Additional preparedness and precautions are ne...,The investigation board recommended correcting...,None,N/A,None,"Aeronautics Research, Science, Exploration S...",None
3,4,1991-04-19,jsc,Flight Hardware Damage (Tires) due to Ground T...,,"On November 21, 1989 two shuttle orbiter tires...",Lack of fidelity between ground test system co...,The test stand configuration was changed to op...,None,N/A,None,"Aeronautics Research, Human Exploration and Op...",None
4,5,1991-04-21,jsc,Inadequate Electrical Cable Strain Relief Desi...,,A pair of shuttle orbiter teleprinter 28 VDC p...,Inadequate strain relief design for electrical...,The orbiter teleprinter cable strain relief de...,None,N/A,None,"Aeronautics Research, Science, Exploration S...",None
5,6,1991-04-28,jsc,Orbiter Exterior Panel Separation During Fligh...,,"Following the shuttle mission STS-27, the righ...",Incomplete assembly or installation instructio...,Procedures were changed to include detailed in...,None,N/A,None,"Aeronautics Research, Science, Exploration S...",None
6,7,1991-04-29,jsc,Debonding of a Mounting Device (Shuttle Orbite...,,During two successive missions of the shuttle ...,Deformation or growth of base material (wall o...,Stiffeners were added to a section of the crew...,None,N/A,None,Human Exploration and Operations,None
7,8,1991-06-10,jsc,Hazard Criticality in the Design and Manufactu...,,A flash fire occurred during a performance rec...,Failure to minimize flow passage configuration...,The mishap investigation board recommendations...,None,N/A,None,Human Exploration and Operations,None
8,9,1991-06-13,jsc,Lubricant Particles Clogging Spacecraft System...,,Low flow rates were observed in the shuttle or...,Use of assembly lubricant materials with suspe...,Contaminated system components were disassembl...,None,N/A,None,"Aeronautics Research, Science, Exploration S...",None
9,10,1991-06-17,jsc,Repair of a Failed Component (Shuttle Orbiter ...,,A shuttle orbiter General Purpose Computer (GP...,Repairing a failed component (GPC IOP Soldered...,Rework procedures were changed to provide an i...,None,N/A,None,"Aeronautics Research, Human Exploration and Op...",None


In [8]:

#csv_file = "Raw_Lessons_Learned_text.csv"
csv_file = "test_Lessons_Learned_text.csv"
lessons_learned.to_csv(csv_file)